# Multi-Factor Investing 

In [26]:
import pandas as pd
import requests
import json
from PIL import Image
import transformers
import os
import datetime
from lxml import etree
import sqlite3

### I Data Import by Webscraping

### II Data Analysis